In [44]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
tf.__version__

'2.7.0'

# Dutch to Spanish

## There is no Dutch to Spanish database available in manythings.org, so we will create a Dutch to English, English to Spanish model.

In [45]:
# Download data
!curl -O http://www.manythings.org/anki/nld-eng.zip
!unzip nld-eng.zip
data_path = "nld.txt" # Path to the data txt file on disk.

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2059k  100 2059k    0     0  5597k      0 --:--:-- --:--:-- --:--:-- 5597k
Archive:  nld-eng.zip
replace _about.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: _about.txt              
replace nld.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: nld.txt                 


In [46]:
# Configuration
batch_size = 64
epochs = 15
latent_dim = 256

# length of dataset
num_samples = 20000

# Read in data
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

# The *with* statement is a handy way of managing file I/O. It ensures that
# the file gets closed once we are done with it.
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")

for line in lines[: min(num_samples, len(lines) - 1)]:
    target_text, input_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)
f.close()

len(input_texts), len(target_texts)


(20000, 20000)

In [47]:
# Take a peek at the first few pairs of input and target texts
[ (i, t) for i, t in enumerate(zip(input_texts[:15], target_texts[:15]))]

[(0, ('Lopen!', '\tGo.\n')),
 (1, ('Vooruit.', '\tGo.\n')),
 (2, ('Hoi.', '\tHi.\n')),
 (3, ('Hé!', '\tHi.\n')),
 (4, ('Hai!', '\tHi.\n')),
 (5, ('Ren!', '\tRun!\n')),
 (6, ('Vlucht.', '\tRun!\n')),
 (7, ('Ren!', '\tRun.\n')),
 (8, ('Vlucht.', '\tRun.\n')),
 (9, ('Wie?', '\tWho?\n')),
 (10, ("Da's niet gek!", '\tWow!\n')),
 (11, ('Eend!', '\tDuck!\n')),
 (12, ('Vuur!', '\tFire!\n')),
 (13, ('Brand!', '\tFire!\n')),
 (14, ('Help!', '\tHelp!\n'))]

In [48]:
# find the length of the maximum sequece for input and ouput
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

# sort list of unique characters for each input and get their lengths
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

Max sequence length for inputs: 52
Max sequence length for outputs: 24


# Vectorize the input and target phrases

In [49]:
# Make Python dictionaries for the input and target messages
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

# The encoder inputs - will hold character sequences for Dutch phrases
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32")

# The decoder inputs - will hold character sequences for English phrases
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")

# Same size numpy array for the decoder targets
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")

# Replace the characters in the input and output phrases with the vectors

In [50]:
# Iterate over all of our phrase pairs
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    
    # Iterate over all of the characters in the input phrase
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0

    # This adds padding with spaces
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0

    # Iterate over all of the characters in the target phrase. Here we are
    # filling two vectors 
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    
    # This adds padding with spaces
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0


# Build the Encoder Model


In [51]:
# 1. Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))

# 2. Use a LSTM layer to process the input vectors. After today's lecture
# you should know what return_state does.
encoder = keras.layers.LSTM(latent_dim, return_state=True)

# 3. Save the output from the encoder, but see step 4. 
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# 4. We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]



# This takes the target tokens as the input.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# The LSTM later has the same internal dimensionality as for the encoder.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)

# Save the decoder output: Note that this uses decoder_inputs
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

# Dense with softmax allows us to predict categorical output (our list of French characters)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")

# Output layer
decoder_outputs = decoder_dense(decoder_outputs)

# Define the overall model. This binds the encoder and decoder and will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model 

In [55]:
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["categorical_accuracy"])

model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, None, 81)]   0           []                               
                                                                                                  
 input_6 (InputLayer)           [(None, None, 76)]   0           []                               
                                                                                                  
 lstm_2 (LSTM)                  [(None, 256),        346112      ['input_5[0][0]']                
                                 (None, 256),                                                     
                                 (None, 256)]                                                     
                                                                                            

# Train the Dutch to English model

In [53]:
epochs = 10

history = model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,)


Epoch 1/10
250/250 [==============================] - 14s 43ms/step - loss: 2.0660 - categorical_accuracy: 0.4479 - val_loss: 2.2479 - val_categorical_accuracy: 0.3594
Epoch 2/10
250/250 [==============================] - 9s 34ms/step - loss: 1.4817 - categorical_accuracy: 0.5697 - val_loss: 1.9999 - val_categorical_accuracy: 0.4234
Epoch 3/10
250/250 [==============================] - 8s 34ms/step - loss: 1.2946 - categorical_accuracy: 0.6183 - val_loss: 1.7401 - val_categorical_accuracy: 0.4907
Epoch 4/10
250/250 [==============================] - 8s 34ms/step - loss: 1.1782 - categorical_accuracy: 0.6498 - val_loss: 1.6271 - val_categorical_accuracy: 0.5147
Epoch 5/10
250/250 [==============================] - 9s 34ms/step - loss: 1.0994 - categorical_accuracy: 0.6710 - val_loss: 1.5875 - val_categorical_accuracy: 0.5322
Epoch 6/10
250/250 [==============================] - 8s 34ms/step - loss: 1.0369 - categorical_accuracy: 0.6887 - val_loss: 1.5737 - val_categorical_accuracy: 0.53

# Construct the encoder and decoder

In [56]:
# Define sampling models

encoder_inputs = model.input[0]  # Encoder input layer
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # First LSTM

# This is the "thought vector" the hidden state that is used to start the decoder
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # This is the second/decoder input layer
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_lstm = model.layers[3] # Second LSTM 

decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]

decoder_dense = model.layers[4] # Dense output layer

decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

In [57]:
# Make Python dictionaries that map vectors to characters
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [68]:
# Decode sequences. convert a one-hot encoded input sequence
# and converts it into a 

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    #start = time.process_time()
    states_value = encoder_model.predict(input_seq)
    #print("Encoding:", time.process_time() - start)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    stop_condition = False
    decoded_sentence = ""
    
    # Always a little risky to use a while loop, but we don't know what 
    # length of sentence the decoder will issue - that's pretty much the 
    # whole point of a sequence-to-sequence model, right?
    while not stop_condition:
        #start = time.process_time()
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        #print("Decoding:", time.process_time() - start)
        # Sample a token: Find the index of the most probable output character
        sampled_token_index = np.argmax(output_tokens[0, -1, :])

        # Use our reversing dictionary to decode the predicted character
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length of a decoder output sequence
        # or find that the decoder has issued a stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))

        # This one-hot encodes the current character to use as input for the next iteration
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    
    return decoded_sentence

In [73]:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]

    # Here's where we call our custom function
    decoded_sentence = decode_sequence(input_seq)

    print("Input sentence:", input_texts[seq_index], "Decoded sentence:", decoded_sentence)
    

Input sentence: Lopen! Decoded sentence: Don't sell you sone?

Input sentence: Vooruit. Decoded sentence: Let's see that book.

Input sentence: Hoi. Decoded sentence: How did you stall?

Input sentence: Hé! Decoded sentence: He was stall a lot.

Input sentence: Hai! Decoded sentence: It's still to go.

Input sentence: Ren! Decoded sentence: Let's see that book.

Input sentence: Vlucht. Decoded sentence: Let's see that book.

Input sentence: Ren! Decoded sentence: Let's see that book.

Input sentence: Vlucht. Decoded sentence: Let's see that book.

Input sentence: Wie? Decoded sentence: Who is that wanter?

Input sentence: Da's niet gek! Decoded sentence: That's a pine car.

Input sentence: Eend! Decoded sentence: Are you strenge?

Input sentence: Vuur! Decoded sentence: Let's see that book.

Input sentence: Brand! Decoded sentence: Are you strenge?

Input sentence: Help! Decoded sentence: My dog is stall.

Input sentence: Spring. Decoded sentence: Let's see that book.

Input sentence: 

In [70]:

def text_encoder(my_phrase):
  if len(my_phrase) <= max_encoder_seq_length:
      # Fill up a single one-hot encoding vector with zeroes
      my_input_data =  np.zeros((1, max_encoder_seq_length, num_encoder_tokens), dtype="float32")

      # Iterate over all of the characters in the input phrase
      for t, char in enumerate(my_phrase):
          my_input_data[0, t, input_token_index[char]] = 1.0

      # This adds padding with spaces
      my_input_data[0, t + 1 :, input_token_index[" "]] = 1.0

      return my_input_data
  else:
      print("Input phrase is longer than the maximum encoder sequence length.")

In [59]:

# save list to txt
with open('/content/drive/MyDrive/NLP/hw5/english_reference_vector.txt', 'w') as f:
  for phrase in english_reference:
    f.write('%s\n' % phrase)


FileNotFoundError: ignored

# English to Spanish Model

In [ ]:
# read in english to Spanish
# Download data
!curl -O http://www.manythings.org/anki/spa-eng.zip
!unzip spa-eng.zip
data_path_spa = "spa.txt" # Path to the data txt file on disk.


In [ ]:
# Configuration
batch_size = 64
epochs = 15
latent_dim = 256

# length of dataset
num_samples = 20000

# Read in data
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

# The *with* statement is a handy way of managing file I/O. It ensures that
# the file gets closed once we are done with it.
with open(data_path_spa, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")

for line in lines[: min(num_samples, len(lines) - 1)]:
    target_text, input_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

print(len(input_texts), len(target_texts))
# Take a peek at the first few pairs of input and target texts
[ (i, t) for i, t in enumerate(zip(input_texts[:15], target_texts[:15]))]

In [30]:
# find the length of the maximum sequece for input and ouput
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

# sort list of unique characters for each input and get their lengths
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)